In [22]:
import numpy as np
import pandas as pd
import geopandas as gpd
import random
import json
from geopy.distance import geodesic
from pyscipopt import Model, quicksum, multidict

In [80]:
pu = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/GIS_files/hs_full_geo.geojson').set_index('pu_2324_84')
pu = pu.to_crs('EPSG:4326')

In [82]:
pu_half_SGR = pu.copy()
pu_half_SGR['basez+gen'] = pu['basez'] + 0.3*pu['student_gen']

pu_data = pu_half_SGR['basez+gen'].to_dict()
I, d = multidict(pu_data)

In [84]:
CFLP_solution = gpd.read_file('/Users/leahwallihan/Durham_school_planning/DPS-Planning/CFLP/CFLP_sols/noDSA/CFLP_hs_fullSGR2.geojson')

In [86]:
with open('/Users/leahwallihan/Durham_school_planning/DPS-Planning/CFLP/CFLP_sols/noDSA/CFLP_hs_fullSGR.json', 'r') as f:
    solution_reports = json.load(f)

In [90]:
solution_report = solution_reports[1]
new_boundary = solution_report['assignments']['1']

In [102]:
centroid_coords = {
    idx: (geom.y, geom.x)  # (latitude, longitude)
    for idx, geom in pu.geometry.centroid.items()
}

/var/folders/ml/wj07z83j0yq4rjs8n_7sv0mw0000gn/T/ipykernel_55285/1900494518.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  for idx, geom in pu.geometry.centroid.items()


In [104]:
total_distance1 = 0
for geom in new_boundary:
    total_distance1 += geodesic(centroid_coords[1], centroid_coords[geom]).miles * d[geom]

In [106]:
total_distance1

12541.911516322989

In [108]:
total_distance179 = 0
for geom in new_boundary:
    total_distance179 += geodesic(centroid_coords[179], centroid_coords[geom]).miles * d[geom]

In [110]:
total_distance179

7761.467194075424

In [118]:
# from CFLP:
not_central = pu[(pu['Region'] != 'Central')]

# initialize dictionary of planning units with capacity of 1600 for potential site
pu_dict = {}
for idx, row in not_central.iterrows():
    pu_dict[idx] = 1550

J, M = multidict(pu_dict)

c = {}
for i in I:
    for j in J:
        c[i, j] = geodesic(centroid_coords[i], centroid_coords[j]).miles

In [119]:
geodesic(centroid_coords[179], centroid_coords[1]).miles

4.694202218966093

In [122]:
c[179, 1]

4.694202218966093

In [124]:
pu

,X,Y,Region,Shape_Area,count,basez,student_gen,geometry
pu_2324_84,,,,,,,,
1,2.049028e+06,893128.606346,North,3.556198e+07,6.0,6.0,0,"POLYGON ((-78.82257 36.19379, -78.82287 36.192..."
2,2.039923e+06,835434.121317,North,2.080136e+07,18.0,8.0,0,"POLYGON ((-78.86225 36.05178, -78.85945 36.049..."
3,2.058005e+06,797649.028742,East,1.329077e+07,4.0,4.0,0,"POLYGON ((-78.79371 35.94418, -78.79394 35.943..."
4,2.003222e+06,772895.737307,Southwest,4.684140e+07,6.0,6.0,0,"POLYGON ((-78.98659 35.88678, -78.98626 35.886..."
5,2.067334e+06,828244.230730,East,1.497046e+08,6.0,5.0,0,"POLYGON ((-78.7536 36.03136, -78.74349 36.0254..."
...,...,...,...,...,...,...,...,...
847,2.013328e+06,854621.040074,North,3.748151e+06,0.0,0.0,0,"POLYGON ((-78.95124 36.10239, -78.95124 36.102..."
848,2.032661e+06,834741.061398,North,1.033344e+06,25.0,19.0,0,"POLYGON ((-78.88798 36.04156, -78.88814 36.041..."
849,2.040116e+06,820883.548010,North,2.898398e+07,0.0,0.0,0,"POLYGON ((-78.97497 36.05557, -78.97344 36.053..."
